In [1]:
# Implementing Echo State Network for Mackey-Glass Series

In [2]:
# Importing required modules

In [3]:
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import linalg
from ipywidgets import *
from IPython.display import *

In [5]:
# Setting random seed

In [6]:
def set_seed(seed=None):
    """Making the seed (for random values) variable if None"""

    if seed is None:
        import time
        seed = int((time.time()*10**6) % 4294967295)
        print(seed)
    try:
        np.random.seed(seed)
        print("Seed used for random values:", seed)
    except:
        print("!!! WARNING !!!: Seed was not set correctly.")
    return seed

In [7]:
# Creating Network Class

In [8]:
class Network(object):

    def __init__(self, trainLen=2000, testLen=2000, initLen=100) :
        self.initLen = initLen
        self.trainLen = trainLen
        self.testLen = testLen
        self.data = np.loadtxt("MackeyGlass_t17.txt")
        self.inSize = self.outSize = 1 #Input/Output dimensions
        self.resSize = 300 #Reservoir size (prediction)
        #self.resSize = 1000 #Reservoir size (generation)
        self.a = 0.3 #Leak rate alpha
        self.spectral_radius = 1.25 #Spectral raidus
        self.input_scaling = 1. #Input scaling
        self.reg =  1e-8 #None #Regularization factor - if None,
        #we'd use pseudo-inverse rather than ridge regression

        self.mode = 'prediction'
        #self.mode = 'generative'
        
        self.mse = 0

        #Change the seed, reservoir performances should be averaged accross
        #at least 20 random instances (with the same set of parameters)
        seed = None #42

        set_seed(seed)
        
nw = Network()

3607131467
Seed used for random values: 3607131467


In [9]:
# Plotting data sample

In [10]:
def plot_figure(f) :
    plt.figure(0).clear()
    plt.figure(figsize=(10,7))
    plt.plot(nw.data[0:int(f)])
    plt.ylim([-1.1,1.1])
    plt.title('A sample of input data')
    
interact(plot_figure, f=FloatSlider(value=2000,min=1000,max=10000,step=1000,continuous_update=False,description='time steps'))

interactive(children=(FloatSlider(value=2000.0, continuous_update=False, description='time steps', max=10000.0…

<function __main__.plot_figure(f)>

In [11]:
# Generating Win,W randomly and then generating X,Ytarget 

In [12]:
from scipy.sparse import rand

In [13]:
def initialization(nw) :

    #Weights
    nw.Win = np.array(rand(nw.resSize,1+nw.inSize, density=0.25, format="csr", random_state=42).todense())
    nw.W = np.array(rand(nw.resSize,nw.resSize, density=0.25, format="csr", random_state=42).todense())
    
    #Matrices
    #Allocated memory for the design (collected states) matrix
    nw.X = np.zeros((1+nw.inSize+nw.resSize,nw.trainLen-nw.initLen))
    #Set the corresponding target matrix directly
    nw.Ytarget = nw.data[None,nw.initLen+1:nw.trainLen+1]

    #Run the reservoir with the data and collect X
    nw.x = np.zeros((nw.resSize,1))  
    
    return(nw)

We compute <b style="color:#ffcc00">W</b> (reservoir weights) spectral radius (i.e., the biggest of the absolute eigenvalues). This number will then be used to scale the weights : the biggest the spectral radius is, the easiest it will be for the system to remember longer inputs.

In [14]:
# Computing spectral radius(biggest of the absolute eigen values of W matrix) and then scaling W matrix using that

In [15]:
def compute_spectral_radius(nw):
    print('Computing spectral radius...',end=" ")
    rhoW = max(abs(linalg.eig(nw.W)[0]))
    print('Done.')
    nw.W *= nw.spectral_radius / rhoW
    
    return(nw)

In [16]:
# Learning phase

In [17]:
# 𝑥𝑛 = (1−𝛼)𝑥𝑛 − 1 × 𝛼tanh(𝑊𝑖𝑛.𝑢𝑛 − 1) + 𝑊.𝑥𝑛−1

In [18]:
def learning_phase(nw) :
    for t in range(nw.trainLen):
        #Input data
        nw.u = nw.data[t]
        nw.x = (1-nw.a)*nw.x + nw.a*np.tanh( np.dot(nw.Win, np.vstack((1,nw.u)) ) + np.dot( nw.W, nw.x ) )
        #After the initialization, we start modifying X
        if t >= nw.initLen:
            nw.X[:,t-nw.initLen] = np.vstack((1,nw.u,nw.x))[:,0]
            
    return(nw)

In [19]:
# Training output weights using ridge regression
# 𝑊𝑜𝑢𝑡 = (𝑌𝑡.𝑋𝑇) . (𝑋.𝑋𝑇+𝑟𝑒𝑔.𝐼)^-1

In [20]:
def train_output(nw) :
    nw.X_T = nw.X.T
    if nw.reg is not None:
        # Ridge regression (linear regression with regularization)
        nw.Wout = np.dot(np.dot(nw.Ytarget,nw.X_T), linalg.inv(np.dot(nw.X,nw.X_T) + \
            nw.reg*np.eye(1+nw.inSize+nw.resSize) ) )
    else:
        # Pseudo-inverse
        nw.Wout = np.dot(nw.Yt, linalg.pinv(nw.X) )
        
    return(nw)

In [21]:
# Testing in a particular mode

In [22]:
def test(nw) :
    #Run the trained ESN in a generative mode. no need to initialize here, 
    #because x is initialized with training data and we continue from there.
    nw.Y = np.zeros((nw.outSize,nw.testLen))
    nw.u = nw.data[nw.trainLen]
    for t in range(nw.testLen):
        nw.x = (1-nw.a)*nw.x + nw.a*np.tanh( np.dot(nw.Win, np.vstack((1,nw.u)) ) + np.dot(nw.W,nw.x ) )
        nw.y = np.dot(nw.Wout, np.vstack((1,nw.u,nw.x)) )
        nw.Y[:,t] = nw.y
        if nw.mode == 'generative':
            #Generative mode:
            nw.u = nw.y
        elif nw.mode == 'prediction':
            #Predictive mode:
            nw.u = nw.data[nw.trainLen+t+1] 
        else:
            raise(Exception, "ERROR: 'mode' was not set correctly.")
    
    return(nw)

In [23]:
def compute_error(nw) :
    # Computing MSE for the first errorLen iterations
    errorLen = 500
    nw.mse = sum( np.square( nw.data[nw.trainLen+1:nw.trainLen+errorLen+1] - nw.Y[0,0:errorLen] ) ) / errorLen
    print('MSE = ' + str( nw.mse ))
    
    return(nw)

In [24]:
def compute_network(nw) :
    nw = initialization(nw)
    nw = compute_spectral_radius(nw)
    nw = learning_phase(nw)
    nw = train_output(nw)
    nw = test(nw)  
    nw = compute_error(nw)
    return(nw)

In [25]:
# Definition of the network parameters

In [34]:
select_mode = ToggleButtons(description='Mode:',
    options=['prediction', 'generative'])
var1 = FloatSlider(value=400, min=0, max=1000, step=1, description='resSize')
var2 = FloatSlider(value=100, min=0, max=2000, step=1, description='initLen')
var3 = FloatSlider(value=2000, min=0, max=5000, step=1, description='trainLen')
var4 = FloatSlider(value=2000, min=0, max=8000, step=500, description='testLen')
var5 = FloatSlider(value=1.25, min=0, max=10, step=0.01, description='spectral radius')
var6 = FloatSlider(value=0.3, min=0, max=1, step=0.01, description='leak rate')
valid = Button(description='Validate')

def record_values(_) :
    clear_output()
    nw.mode=select_mode.value
    nw.resSize=int(var1.value)
    nw.initLen=int(var2.value)
    nw.trainLen=int(var3.value)
    nw.testLen=int(var4.value)
    nw.spectral_radius=float(var5.value)
    nw.a=float(var6.value)
    print("InitLen:", nw.initLen, "TrainLen:", nw.trainLen, "TestLen:", nw.testLen) 
    print("ResSize:", nw.resSize, "Spectral Radius:", nw.spectral_radius, "Leak Rate:", nw.a)
    compute_network(nw)
    return(nw)


display(select_mode)
display(var1)
display(var2)
display(var3)
display(var4)
display(var5)
display(var6)
display(valid)

valid.on_click(record_values)   

InitLen: 100 TrainLen: 2000 TestLen: 2000
ResSize: 400 Spectral Radius: 1.25 Leak Rate: 1.0
Computing spectral radius... Done.
MSE = 1.4488097770663885e-06


In [ ]:
# Graph 1: Comparison between expected and estimated outputs

In [ ]:
var7 = FloatSlider(value=2000,min=10,max=10000,step=10,description='time steps')
valid = Button(description='Validate')
        
def trace_graph1(_) :
    clear_output()
    f=int(var7.value)
    plt.figure(1).clear()
    plt.figure(figsize=(12,7))
    plt.plot( nw.data[nw.trainLen+1:nw.trainLen+f+1], 'g' )
    plt.plot( nw.Y.T[0:f], 'b' )
    plt.title('Target and generated signals $y(n)$ starting at $n=0$')
    if nw.mode == 'generative':
        plt.legend(['Target signal', 'Free-running predicted signal'])
    elif nw.mode == 'prediction':
        plt.legend(['Target signal', 'Predicted signal'])
    
valid.on_click(trace_graph1)
    
display(var7)
display(valid)

In [ ]:
# Graph 2: Difference between expected and estimated outputs

In [ ]:
var8 = FloatSlider(value=2000,min=10,max=nw.testLen,step=10,description='time steps')
var9 = FloatSlider(value=0.2,min=0.1,max=10,step=0.001,description='amplitude')
valid = Button(description='Validate')
        
def trace_graph2(_) :
    clear_output()
    f=int(var8.value)
    amp=float(var9.value)
    plt.figure(2).clear()
    plt.figure(figsize=(12,7))
    plt.ylim([-amp,amp])
    plt.plot(nw.data[nw.trainLen+1:nw.trainLen+f+1]-nw.Y[0][0:f], 'g' )
    print(nw.Y[0].shape)
    plt.title('Target and predicted signal difference through time')
    if nw.mode == 'generative':
        plt.legend(['Target signal', 'Free-running predicted signal'])
    elif nw.mode == 'prediction':
        plt.legend(['Target signal', 'Predicted signal'])
    
valid.on_click(trace_graph2)
    
display(var8)
display(var9)
display(valid)

In [ ]:
# Graph 3: Plotting neurons activations (total)

In [ ]:
var10 = FloatSlider(value=2000,min=10,max=nw.trainLen-nw.initLen,step=10,description='time steps')
var11 = FloatSlider(value=10, min=1, max=nw.resSize, step=1, description='number of neurons')
valid = Button(description='Validate')

def trace_graph3(_) :
    clear_output()
    f=int(var10.value)
    nb=int(var11.value)
    plt.figure(3).clear()
    plt.figure(figsize=(10,7))
    plt.plot( nw.X[2:2+nb,0:f].T )
    print(nw.X.shape)
    plt.ylim([-1.1,1.1])
    plt.title('Activations $\mathbf{x}(n)$ from Reservoir Neurons ID 0 to '+str(nb-1)+' for '+str(f)+' time steps')
    
valid.on_click(trace_graph3)
    
display(var10)
display(var11)
display(valid)

In [ ]:
nw.X.shape

In [ ]:
# Graph 4: Plotting single neuron activation

In [ ]:
var12 = FloatSlider(value=2000,min=10,max=nw.trainLen-nw.initLen,step=10,description='time steps')
var13 = FloatSlider(value=2, min=0, max=nw.resSize-1, step=1, description='neuron ID')
valid = Button(description='Validate')

def trace_graph4(_) :
    clear_output()
    f=int(var12.value)
    num=int(var13.value)
    plt.figure(4).clear()
    plt.figure(figsize=(10,5))
    plt.plot( nw.X[2+num,:f].T )
    plt.ylim([-1.1,1.1])
    plt.title('Activations $\mathbf{x}(n)$ from Reservoir Neuron ID '+str(num)+' for '+str(f)+' time steps')

valid.on_click(trace_graph4)

display(var12)
display(var13)
display(valid)

In [ ]:
# Graph 5: Output weights at the end of the simulation

In [ ]:
valid = Button(description='Show')

def trace_graph5(_) :
    clear_output()
    plt.figure(5).clear()
    plt.figure(figsize=(12,7))
    plt.bar(range(1+nw.inSize+nw.resSize), np.squeeze(nw.Wout.T) )
    plt.title('Output weights $\mathbf{W}^{out}$')

valid.on_click(trace_graph5)

display(valid)


In [ ]:
nw.Wout.T.shape